In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt

In [310]:
def clean_names(s):
    return pd.Series(s).str.strip().str.replace("[^0-9a-zA-Z ]+", "").str.replace(' ', "_").str.lower().values

class FinancialAnalysis:
    def __read_income_statement(self, file):
        # Read input
        results = pd.read_excel(file, header = 0, skiprows = 14, sheet_name = "Income Statement")
        results = results.iloc[1:,]  # Remove currency row

        # Clean dates
        results.columns = results.columns.map(lambda x: re.search("[a-zA-Z]{3}-\d{1,2}-\d{4}|$", x)[0])

        # Remove blank rows
        clean_rownames = results.iloc[:,0].map(lambda x: str(x).strip())
        valid_rows = ~((clean_rownames == "nan") | (clean_rownames == ""))
        results = results[valid_rows]

        # Find end of income statement
        clean_rownames = results.iloc[:,0].map(lambda x: str(x).strip())
        last_row = np.where(clean_rownames == "Net Income")[0].item()
        results = results.iloc[:(last_row+1),]

        # Transpose index
        results = results.set_index("").transpose()

        # Set index as date
        results.index = pd.to_datetime(results.index).date

        # Replace - with zero and convert to numeric
        results = results.replace({"-":"0"}).apply(pd.to_numeric)
        self.income_statement = results
        self.inc_st_meta = pd.DataFrame({"fields": self.income_statement.columns.values}).set_index("fields")
        
    def __read_balance_sheet(self, file):
        results = pd.read_excel(file, header = 0, skiprows = 14, sheet_name = "Balance Sheet")
        results = results.iloc[1:,]  # Remove currency row
        #print(results)
        # Clean dates
        results.columns = results.columns.map(lambda x: re.search("[a-zA-Z]{3}-\d{1,2}-\d{4}|$", x)[0] if isinstance(x, str) else x)

        # Remove blank rows
        clean_rownames = results.iloc[:,0].map(lambda x: str(x).strip())
        valid_rows = ~((clean_rownames == "nan") | (clean_rownames == ""))
        results = results[valid_rows]

        # Find end of income statement
        clean_rownames = results.iloc[:,0].map(lambda x: str(x).strip())
        last_row = np.where(clean_rownames == "Total Liabilities And Equity")[0].item()
        results = results.iloc[:(last_row+1),]
        
        # Transpose index
        results = results.set_index("").transpose()

        # Set index as date
        results.index = pd.to_datetime(results.index).date

        # Replace - with zero and convert to numeric
        results = results.replace({"-":"0"}).apply(pd.to_numeric).dropna(axis = 1, how = 'all')

        self.balance_sheet = results
    
    def read_xls(self, file):
        self.__read_income_statement(file)
        self.__read_balance_sheet(file)
        
    def read_segment(self, file, sheet = "Segments"):
        results = pd.read_excel(file, header = 0, skiprows = 4, sheet_name = sheet, index_col = 0)
        
        #results = pd.read_excel("std_mastercard.xls", header = 0, skiprows = 4, sheet_name = "Segments_bbg", index_col = 0)
        results = results.dropna(axis = 1, how = 'all').transpose()
        results.index = pd.to_datetime(results.index, format = "%m/%d/%Y")

        segments = pd.DataFrame({"has_negatives": (results < 0).any()})
        segments['subcategory'] = np.where(segments['has_negatives'], "contra_rev", "op_rev")
        segments['category'] = "rev"
        segments['item'] = clean_names(segments.index)

        segments = segments.drop('has_negatives', axis = 1)
        results.columns = segments.item
        
        # Build multilevel column index
        multilevel_header = pd.MultiIndex.from_frame(
            pd.DataFrame({'item':results.columns}).set_index("item").join(  # Build df using income_statement columns
                segments.set_index("item").dropna()).reset_index().iloc[:,-1::-1])   # Add other fields
        results.columns = multilevel_header

        self.inc_st_meta = pd.concat([self.inc_st_meta[self.inc_st_meta.category != "rev"], segments], axis = 0, sort = False)
        self.income_statement = self.income_statement.drop('op_rev', axis = 1, level = 'subcategory').join(results)
        
    def __assoc_match(self, df1, df2):
        # Search for unmatched columns in df1 to find matches in df2, returns df1 where filled
        filled_rows = df1.dropna()
        unfilled_rows = df1[np.isnan(df1)]
        return pd.concat([filled_rows,
                         unfilled_rows.join(df2)])
    
    def __replace_nas(self, df, df_search):
        return df.combine_first(df.drop(columns = df.columns).join(df_search))
    
    def __categorize_bs(self):
        cat_asset = pd.DataFrame((
            ("current", "op", "Cash And Equivalents"), 
            ("current", "op", "Short Term Investments"),
            ("current", "op", "Accounts Receivable"),
            ("current", "op", "Prepaid Exp."), 
            ("current", "op", "Deferred Tax Assets, Curr."), 
            ("current", "op", "Restricted Cash"),
            ("current", "op", "Other Current Assets"), 
            ("noncurrent", "op", "Gross Property, Plant & Equipment"), 
            ("noncurrent", "op", "Accumulated Depreciation"), 
            ("noncurrent", "fin", "Long-term Investments"),
            ("noncurrent", "op", "Goodwill"), 
            ("noncurrent", "op", "Other Intangibles"), 
            ("noncurrent", "op", "Deferred Tax Assets, LT"), 
            ("noncurrent", "op", "Deferred Charges, LT"), 
            ("noncurrent", "op", "Other Long-Term Assets")), columns = ["horizon", "opfin", "statement"])
        cat_asset["aloe"] = "asset"
        
        cat_liability = pd.DataFrame((
            ("current", "op", "Accounts Payable"), 
            ("current", "op", "Accrued Exp."), 
            ("current", "op", "Curr. Port. of LT Debt"), 
            ("current", "op", "Curr. Port. of Leases"), 
            ("current", "op", "Curr. Income Taxes Payable"), 
            ("current", "op", "Unearned Revenue, Current"),
            ("current", "op", "Other Current Liabilities"), 
            ("noncurrent", "fin", "Long-Term Debt"), 
            ("noncurrent", "op", "Long-Term Leases"), 
            ("noncurrent", "op", "Unearned Revenue, Non-Current"), 
            ("noncurrent", "op", "Pension & Other Post-Retire. Benefits"), 
            ("noncurrent", "op", "Def. Tax Liability, Non-Curr."), 
            ("noncurrent", "op", "Other Non-Current Liabilities")), columns = ["horizon", "opfin", "statement"])
        cat_liability["aloe"] = "liability"
            
        cat_eq = pd.DataFrame(["Common Stock", "Additional Paid In Capital", "Retained Earnings", "Treasury Stock", \
                               "Comprehensive Inc. and Other", "Minority Interest"], columns = ["statement"])
        cat_eq['opfin'] = cat_eq['horizon'] = cat_eq['aloe'] = "eq"
        cat_eq = cat_eq[cat_asset.columns]
        
        cat_all = pd.concat([cat_asset, cat_liability, cat_eq]).set_index("statement")
        cat_all['item'] = clean_names(cat_all.index.values)
        
        self.bs_meta = cat_all
        
    
    def __categorize_inc_st(self):
        sub_cat_rev = pd.DataFrame((
            ("op_rev", "core_revenue", "Revenue"),
            ("op_rev", "non_core_revenue", "Other Revenue"), 
            ("fin_rev", "interest_income", "Interest and Invest. Income")), columns = ["subcategory", "item", "statement"])
        sub_cat_rev["category"] = "rev"
        sub_cat_rev = sub_cat_rev.set_index("statement")
        self.inc_st_meta = self.__replace_nas(self.inc_st_meta, sub_cat_rev)
        
        sub_cat_exp = pd.DataFrame((
            ("op_exp", "cost_of_good_sold", "Cost Of Goods Sold"),         
            ("op_exp", "selling_gen_admin", "Selling General & Admin Exp."),
            ("op_exp", "research_dev", "R & D Exp."),
            ("op_exp", "depreciation", "Depreciation & Amort."),
            ("op_exp", "other_expenses", "Other Operating Expense/(Income)"),
            ("op_exp", "other_non_op", "Other Non-Operating Inc. (Exp.)"),
            ("fin_exp", "interest_expense", "Interest Expense"),
            ("one_off_exp", "restructuring", "Restructuring Charges"), 
            ("one_off_exp", "impairment", "Impairment of Goodwill"), 
            ("one_off_exp", "gain_on_investments", "Gain (Loss) On Sale Of Invest."), 
            ("one_off_exp", "legal_settlement", "Legal Settlements"),
            ("one_off_exp", "other_one_off", "Other Unusual Items"),
            ("tax", "income_tax", "Income Tax Expense")), columns = ["subcategory", "item", "statement"])
        sub_cat_exp["category"] = "exp"
        sub_cat_exp = sub_cat_exp.set_index("statement")

        self.inc_st_meta = self.__replace_nas(self.inc_st_meta, sub_cat_exp)
                             
    def clean_income_statement(self):
        self.__categorize_inc_st()
        self.income_statement = self.income_statement[self.inc_st_meta.dropna().index.values]
        
        # Remove columns with all zeros
        has_value = mastercard.income_statement.apply(abs).sum(axis = 0) != 0
        self.income_statement = self.income_statement[self.income_statement.columns[has_value]]
        
        # Use standardized names for columns
        standardized_columns = pd.DataFrame({"statement": self.income_statement.columns.values}).set_index("statement").join(self.inc_st_meta)['item'].values
        self.income_statement.columns = standardized_columns
        
        # Ensure expense and tax signs are correct
        exp_columns = np.concatenate([self.inc_st_meta[(self.inc_st_meta.subcategory == "op_exp") & (self.inc_st_meta.item != "other_non_op")]['item'].values,
                                      self.inc_st_meta[self.inc_st_meta.subcategory == "tax"]['item'].values])
        found_columns = self.income_statement.columns.intersection(exp_columns)
        self.income_statement[found_columns] = self.income_statement[found_columns].apply(lambda x: abs(x)* -1)
        
        # Build multilevel column index
        multilevel_header = pd.MultiIndex.from_frame(
            pd.DataFrame({'item':self.income_statement.columns}).set_index("item").join(  # Build df using income_statement columns
                self.inc_st_meta.set_index("item").dropna()).reset_index().iloc[:,-1::-1])   # Add other fields
        self.income_statement.columns = multilevel_header
        
    def clean_balance_sheet(self):
        self.__categorize_bs()
        self.balance_sheet = self.balance_sheet[self.bs_meta.dropna().index.values]
        
        # Remove columns with all zeros
        has_value = mastercard.balance_sheet.apply(abs).sum(axis = 0) != 0
        self.balance_sheet = self.balance_sheet[self.balance_sheet.columns[has_value]]
        
        # Use standardized names for columns
        standardized_columns = pd.DataFrame({"statement": self.balance_sheet.columns.values}).set_index("statement").join(self.bs_meta)['item'].values
        self.balance_sheet.columns = standardized_columns
        
        # Build multilevel column index
        multilevel_header = pd.MultiIndex.from_frame(
            pd.DataFrame({'item':self.balance_sheet.columns}).set_index("item").join(  # Build df using income_statement columns
                self.bs_meta.set_index("item").dropna()).reset_index().iloc[:,-1::-1])   # Add other fields
        self.balance_sheet.columns = multilevel_header
        
        # Ensure liability signs are correct
        #self.balance_sheet['liability'] = self.balance_sheet['liability'] * -1
        
        
    def yoy(self):
        return self.income_statement.pct_change()
    
    def get_op_revenue(self, include_contra = True):
        if include_contra:
            return self.income_statement.xs("contra_rev", axis=1, level="subcategory").join(self.income_statement.xs("op_rev", axis=1, level="subcategory")).sum(axis = 1)
        return self.income_statement.xs("op_rev", axis=1, level="subcategory").sum(axis = 1)
    
    def get_op_expense(self):
        return self.income_statement.xs("op_exp", axis=1, level="subcategory").sum(axis = 1)
    
    def get_asset(self): return self.balance_sheet.groupby(level = 'aloe', axis = 1).sum(axis=0)['asset']
    def get_liability(self): return self.balance_sheet.groupby(level = 'aloe', axis = 1).sum(axis=0)['liability']
    def get_equity(self): return self.balance_sheet.groupby(level = 'aloe', axis = 1).sum(axis=0)['eq']

In [329]:
mastercard = FinancialAnalysis()
mastercard.read_xls("data/std_Mastercard.xls")
mastercard.clean_income_statement()
mastercard.clean_balance_sheet()
mastercard.read_segment("data/std_Mastercard.xls", "Segments_bbg")
mastercard.income_statement

category                  exp                                           rev  \
subcategory            op_exp                       fin_exp         fin_rev   
item        selling_gen_admin depreciation interest_expense interest_income   
2014-12-31              -3927         -321              -48              28   
2015-12-31              -4083         -366              -61              25   
2016-12-31              -4526         -373              -95              43   
2017-12-31              -5252         -436             -154              56   
2018-12-31              -6054         -459             -186             122   
2019-12-31              -6697         -522             -224              97   

category             exp                                                     \
subcategory       op_exp   one_off_exp                                        
item        other_non_op restructuring gain_on_investments legal_settlement   
2014-12-31            -7           -87                   0                0   
2015-12-31           -84             0                   0              -61   
2016-12-31           -62             0                   0             -117   
2017-12-31            -7             0                   0              -15   
2018-12-31           -22             0                   0            -1128   
2019-12-31            27             0                 167                0   

category                                                     rev  \
subcategory                      tax                      op_rev   
item        other_one_off income_tax transaction_processing_fees   
2014-12-31              0      -1462                        4035   
2015-12-31            -79      -1150                        4345   
2016-12-31              0      -1587                        5143   
2017-12-31           -167      -2607                        6188   
2018-12-31            -19      -1345                        7391   
2019-12-31              0      -1613                        8469   

category                                                        \
subcategory                                                      
item        domestic_assessments crossborder_volume_fees other   
2014-12-31                  3967                    3054  1688   
2015-12-31                  4086                    3225  1991   
2016-12-31                  4411                    3568  2431   
2017-12-31                  5130                    4174  2853   
2018-12-31                  6138                    4954  3348   
2019-12-31                  6781                    5606  4124   

category                            
subcategory             contra_rev  
item        rebates_and_incentives  
2014-12-31                   -3303  
2015-12-31                   -3980  
2016-12-31                   -4777  
2017-12-31                   -5848  
2018-12-31                   -6881  
2019-12-31                   -8097

In [330]:
mastercard.get_equity()

2014-12-31    6824
2015-12-31    6062
2016-12-31    5684
2017-12-31    5568
2018-12-31    5489
2019-12-31    5991
Name: eq, dtype: int64

In [327]:
mastercard.balance_sheet.divide(mastercard.get_op_revenue(include_contra = True), axis = 0)

aloe                      asset                                             \
opfin                        op                                              
horizon                 current                                              
item       cash_and_equivalents short_term_investments accounts_receivable   
2014-12-31             0.544116               0.131130            0.225188   
2015-12-31             0.594497               0.102514            0.219717   
2016-12-31             0.623701               0.149777            0.230141   
2017-12-31             0.474754               0.147956            0.267104   
2018-12-31             0.446957               0.113445            0.316254   
2019-12-31             0.413907               0.040751            0.326305   

aloe                                                             \
opfin                                                             
horizon                                                           
item       prepaid_exp deferred_tax_assets_curr restricted_cash   
2014-12-31    0.000000                 0.031776        0.057197   
2015-12-31    0.000000                 0.000000        0.055964   
2016-12-31    0.000000                 0.000000        0.050390   
2017-12-31    0.000000                 0.000000        0.043690   
2018-12-31    0.000000                 0.000000        0.036990   
2019-12-31    0.104425                 0.000000        0.034591   

aloe                                                             \
opfin                                                             
horizon                                              noncurrent   
item       other_current_assets gross_property_plant__equipment   
2014-12-31             0.175405                        0.111429   
2015-12-31             0.163546                        0.120617   
2016-12-31             0.173534                        0.123979   
2017-12-31             0.170521                        0.123470   
2018-12-31             0.168027                        0.118261   
2019-12-31             0.081147                        0.108275   

aloe                                                      ...       liability  \
opfin                                                fin  ...              op   
horizon                                       noncurrent  ...      noncurrent   
item       accumulated_depreciation longterm_investments  ... longterm_leases   
2014-12-31                -0.046287             0.025951  ...        0.000000   
2015-12-31                -0.050791             0.017172  ...        0.000000   
2016-12-31                -0.055958             0.012249  ...        0.000000   
2017-12-31                -0.057134             0.019925  ...        0.000000   
2018-12-31                -0.056656             0.022542  ...        0.000268   
2019-12-31                 0.000000             0.000000  ...        0.000000   

aloe                                                                       \
opfin                                                                       
horizon                                                                     
item       unearned_revenue_noncurrent pension__other_postretire_benefits   
2014-12-31                    0.000000                           0.008262   
2015-12-31                    0.000000                           0.006103   
2016-12-31                    0.000000                           0.006403   
2017-12-31                    0.001360                           0.007922   
2018-12-31                    0.006756                           0.005485   
2019-12-31                    0.000000                           0.000000   

aloe                                                               \
opfin                                                               
horizon                                                             
item       def_tax_liability_noncurr other_noncurrent_liabilities   
2014-12-31         

In [328]:
mastercard.balance_sheet

aloe                      asset                                             \
opfin                        op                                              
horizon                 current                                              
item       cash_and_equivalents short_term_investments accounts_receivable   
2014-12-31                 5137                   1238                2126   
2015-12-31                 5747                    991                2124   
2016-12-31                 6721                   1614                2480   
2017-12-31                 5933                   1849                3338   
2018-12-31                 6682                   1696                4728   
2019-12-31                 6988                    688                5509   

aloe                                                             \
opfin                                                             
horizon                                                           
item       prepaid_exp deferred_tax_assets_curr restricted_cash   
2014-12-31           0                      300             540   
2015-12-31           0                        0             541   
2016-12-31           0                        0             543   
2017-12-31           0                        0             546   
2018-12-31           0                        0             553   
2019-12-31        1763                        0             584   

aloe                                                             \
opfin                                                             
horizon                                              noncurrent   
item       other_current_assets gross_property_plant__equipment   
2014-12-31                 1656                            1052   
2015-12-31                 1581                            1166   
2016-12-31                 1870                            1336   
2017-12-31                 2131                            1543   
2018-12-31                 2512                            1768   
2019-12-31                 1370                            1828   

aloe                                                      ...       liability  \
opfin                                                fin  ...              op   
horizon                                       noncurrent  ...      noncurrent   
item       accumulated_depreciation longterm_investments  ... longterm_leases   
2014-12-31                     -437                  245  ...               0   
2015-12-31                     -491                  166  ...               0   
2016-12-31                     -603                  132  ...               0   
2017-12-31                     -714                  249  ...               0   
2018-12-31                     -847                  337  ...               4   
2019-12-31                        0                    0  ...               0   

aloe                                                                       \
opfin                                                                       
horizon                                                                     
item       unearned_revenue_noncurrent pension__other_postretire_benefits   
2014-12-31                           0                                 78   
2015-12-31                           0                                 59   
2016-12-31                           0                                 69   
2017-12-31                          17                                 99   
2018-12-31                         101                                 82   
2019-12-31                           0                                  0   

aloe                                                               \
opfin                                                               
horizon                                                             
item       def_tax_liability_noncurr other_noncurrent_liabilities   
2014-12-31         

In [322]:
mastercard.yoy()

category                  exp                                           rev  \
subcategory            op_exp                       fin_exp         fin_rev   
item        selling_gen_admin depreciation interest_expense interest_income   
2014-12-31                NaN          NaN              NaN             NaN   
2015-12-31           0.039725     0.140187         0.270833       -0.107143   
2016-12-31           0.108499     0.019126         0.557377        0.720000   
2017-12-31           0.160407     0.168901         0.621053        0.302326   
2018-12-31           0.152704     0.052752         0.207792        1.178571   
2019-12-31           0.106211     0.137255         0.204301       -0.204918   

category             exp                                                     \
subcategory       op_exp   one_off_exp                                        
item        other_non_op restructuring gain_on_investments legal_settlement   
2014-12-31           NaN           NaN                 NaN              NaN   
2015-12-31     11.000000          -1.0                 NaN             -inf   
2016-12-31     -0.261905           NaN                 NaN         0.918033   
2017-12-31     -0.887097           NaN                 NaN        -0.871795   
2018-12-31      2.142857           NaN                 NaN        74.200000   
2019-12-31     -2.227273           NaN                 inf        -1.000000   

category                                                     rev  \
subcategory                      tax                      op_rev   
item        other_one_off income_tax transaction_processing_fees   
2014-12-31            NaN        NaN                         NaN   
2015-12-31           -inf  -0.213406                    0.076828   
2016-12-31      -1.000000   0.380000                    0.183659   
2017-12-31           -inf   0.642722                    0.203189   
2018-12-31      -0.886228  -0.484081                    0.194409   
2019-12-31      -1.000000   0.199257                    0.145853   

category                                                            \
subcategory                                                          
item        domestic_assessments crossborder_volume_fees     other   
2014-12-31                   NaN                     NaN       NaN   
2015-12-31              0.029997                0.055992  0.179502   
2016-12-31              0.079540                0.106357  0.220994   
2017-12-31              0.163002                0.169843  0.173591   
2018-12-31              0.196491                0.186871  0.173502   
2019-12-31              0.104757                0.131611  0.231780   

category                            
subcategory             contra_rev  
item        rebates_and_incentives  
2014-12-31                     NaN  
2015-12-31                0.204965  
2016-12-31                0.200251  
2017-12-31                0.224199  
2018-12-31                0.176642  
2019-12-31                0.176719

In [319]:
mastercard.get_op_revenue(), mastercard.get_op_expense()

(2014-12-31     9441
 2015-12-31     9667
 2016-12-31    10776
 2017-12-31    12497
 2018-12-31    14950
 2019-12-31    16883
 dtype: int64, 2014-12-31   -4255
 2015-12-31   -4533
 2016-12-31   -4961
 2017-12-31   -5695
 2018-12-31   -6535
 2019-12-31   -7192
 dtype: int64)

In [320]:
mastercard.get_op_revenue().pct_change()

2014-12-31         NaN
2015-12-31    0.023938
2016-12-31    0.114720
2017-12-31    0.159707
2018-12-31    0.196287
2019-12-31    0.129298
dtype: float64

In [321]:
mastercard.income_statement.divide(mastercard.get_op_revenue(include_contra = False), axis = 0)

category                  exp                                           rev  \
subcategory            op_exp                       fin_exp         fin_rev   
item        selling_gen_admin depreciation interest_expense interest_income   
2014-12-31          -0.308145    -0.025188        -0.003766        0.002197   
2015-12-31          -0.299187    -0.026819        -0.004470        0.001832   
2016-12-31          -0.291005    -0.023983        -0.006108        0.002765   
2017-12-31          -0.286291    -0.023767        -0.008395        0.003053   
2018-12-31          -0.277312    -0.021025        -0.008520        0.005588   
2019-12-31          -0.268094    -0.020897        -0.008967        0.003883   

category             exp                                                     \
subcategory       op_exp   one_off_exp                                        
item        other_non_op restructuring gain_on_investments legal_settlement   
2014-12-31     -0.000549     -0.006827            0.000000         0.000000   
2015-12-31     -0.006155      0.000000            0.000000        -0.004470   
2016-12-31     -0.003986      0.000000            0.000000        -0.007523   
2017-12-31     -0.000382      0.000000            0.000000        -0.000818   
2018-12-31     -0.001008      0.000000            0.000000        -0.051670   
2019-12-31      0.001081      0.000000            0.006685         0.000000   

category                                                     rev  \
subcategory                      tax                      op_rev   
item        other_one_off income_tax transaction_processing_fees   
2014-12-31       0.000000  -0.114721                    0.316620   
2015-12-31      -0.005789  -0.084268                    0.318385   
2016-12-31       0.000000  -0.102038                    0.330676   
2017-12-31      -0.009103  -0.142110                    0.337313   
2018-12-31      -0.000870  -0.061610                    0.338555   
2019-12-31       0.000000  -0.064572                    0.339031   

category                                                            \
subcategory                                                          
item        domestic_assessments crossborder_volume_fees     other   
2014-12-31              0.311284                0.239642  0.132454   
2015-12-31              0.299406                0.236316  0.145893   
2016-12-31              0.283611                0.229409  0.156304   
2017-12-31              0.279640                0.227528  0.155519   
2018-12-31              0.281160                0.226925  0.153360   
2019-12-31              0.271457                0.224420  0.165092   

category                            
subcategory             contra_rev  
item        rebates_and_incentives  
2014-12-31               -0.259181  
2015-12-31               -0.291639  
2016-12-31               -0.307143  
2017-12-31               -0.318779  
2018-12-31               -0.315194  
2019-12-31               -0.324139

2014-12-31    4241
2015-12-31    4365
2016-12-31    4837
2017-12-31    5681
2018-12-31    6491
2019-12-31    7246
Name: op_exp, dtype: int64

In [70]:
def project_item(field, periods, param, df):    
    if param[field]["type"] == "yoy":
        prev_value = df[field][-1]
        values = param[field]["value"].copy()
        if len(values) < periods:
            values = values + [values[-1]] * (periods - len(values))
        elif len(values) > periods:
            values = values[:periods]
            
        projections = np.cumprod(values) * prev_value
        return projections
    
    elif param[field]["type"] == "percentof":
        return 0
        
        
        
    

In [71]:
project_item("Revenue", 5, forecasts, results)

array([18571.3    , 22285.56   , 28971.228  , 37662.5964 , 48961.37532])

In [53]:
forecasts

{'Revenue': {'type': 'yoy', 'value': [1.1, 1.2, 1.3]},
 'Selling General & Admin Exp.': {'type': 'percentof', 'value': 'Revenue'}}

In [54]:
forecasts = {"Revenue":{"type":"yoy",
                        "value":[1.1, 1.2, 1.3]},
            "Selling General & Admin Exp.":{"type": "percentof", 
                                           "value": "Revenue"}}
forecasts

{'Revenue': {'type': 'yoy', 'value': [1.1, 1.2, 1.3]},
 'Selling General & Admin Exp.': {'type': 'percentof', 'value': 'Revenue'}}

In [36]:
forecasts["Revenue"]['value']

[1.2, 1.3]

In [23]:
add_row(results)

,Revenue,Other Revenue,Total Revenue,Cost Of Goods Sold,Gross Profit,Selling General & Admin Exp.,R & D Exp.,Depreciation & Amort.,Other Operating Expense/(Income),"Other Operating Exp., Total",...,NI to Common Incl Extra Items,NI to Common Excl. Extra Items,Per Share Items,Basic EPS,Basic EPS Excl. Extra Items,Weighted Avg. Basic Shares Out.,Diluted EPS,Diluted EPS Excl. Extra Items,Weighted Avg. Diluted Shares Out.,Normalized Basic EPS
2014-12-31,9441.0,0.0,9441.0,0.0,9441.0,3927.0,0.0,321.0,0.0,4248.0,...,3617.0,3617.0,NaN,3.10,3.104721,1165.0,3.10,3.10,1169.0,2.77
2015-12-31,9667.0,0.0,9667.0,0.0,9667.0,4083.0,0.0,366.0,0.0,4449.0,...,3808.0,3808.0,NaN,3.36,3.358024,1134.0,3.35,3.35,1137.0,2.81
2016-12-31,10776.0,0.0,10776.0,0.0,10776.0,4526.0,0.0,373.0,0.0,4899.0,...,4059.0,4059.0,NaN,3.70,3.696721,1098.0,3.69,3.69,1101.0,3.28
2017-12-31,12497.0,0.0,12497.0,0.0,12497.0,5252.0,0.0,436.0,0.0,5688.0,...,3915.0,3915.0,NaN,3.67,3.669165,1067.0,3.65,3.65,1072.0,3.93
2018-12-31,14950.0,0.0,14950.0,0.0,14950.0,6054.0,0.0,459.0,0.0,6513.0,...,5859.0,5859.0,NaN,5.63,5.628242,1041.0,5.60,5.60,1047.0,5.01
2019-12-31,16883.0,0.0,16883.0,0.0,16883.0,6697.0,0.0,522.0,0.0,7219.0,...,8118.0,8118.0,NaN,7.98,7.982300,1017.0,7.94,7.94,1022.0,5.88
2020-12-31,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
